<a href="https://colab.research.google.com/github/d2525456d/Doantotnghiep/blob/main/Thu%E1%BA%ADt%20to%C3%A1n%20m%C3%B4%20ph%E1%BB%8Fng%20luy%E1%BB%87n%20kim/toiuu_luong%E2%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import pandas as pd

# Load the uploaded file
file_path = '/content/data.csv'
df = pd.read_csv(file_path)

# Prepare data
tasks_times = df['TGLT'].tolist()
machines = [
    0, 0, 0, 0, 1, 2, 1, 1, 0, 2, 7, 1, 7, 0, 1, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 4, 1,
    5, 1, 1, 1, 1, 2, 7, 1, 1, 1, 1, 1, 2, 1, 1, 3, 2, 2, 7, 1, 1, 3, 7,
    6, 2, 0, 0, 0, 2, 1, 2, 4, 2, 4, 1, 1, 0, 1, 1, 1, 4, 0, 1, 1, 2, 0, 3, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0
]
df["Thiết bị"] = machines
# Thêm cột "Loại Máy" từ danh sách machines
df['Loại Máy'] = machines

# Thay thế cột "TÊN CÔNG ĐOẠN" bằng chuỗi kết hợp "TÊN CÔNG ĐOẠN" và "Loại Máy"
df['TÊN CÔNG ĐOẠN'] = df['TÊN CÔNG ĐOẠN'] + ' - ' + df['Loại Máy'].astype(str)

proficiency_levels = {
    "Lê Thị Phương Thảo": 1.48, "Hoàng Thị Mỹ Yến": 1.16, "Trần Thị Tuyền": 1.48,
    "Ngô Khắc Hiển": 1.00, "Hồ Thị Diệp": 1.16, "Nguyễn Thị Mỹ Nhung": 1.16,
    "Thái Thị Nga": 1.16, "Trần Thị Cúc": 1.00, "Lê Thị Thanh Hà": 1.16,
    "Nguyễn Thị Thúy": 1.16, "Trần Thị Cẩm Rin": 1.32, "Hoàng Thị Nở": 1.16,
    "Lê Thị Như Quỳnh": 1.16, "Trần Thị Thương": 1.16, "Đinh Thị Mi": 1.16,
    "Nguyễn Thị Ngọc Lan": 1.16, "Bùi Thị Hồng Vân": 1.00, "Trần Thị Oanh": 1.16,
    "Nguyễn Thị Gấm": 1.32, "Phan Thị Thu Hoài": 1.16, "Nguyễn Thị Thu": 1.16,
    "Nguyễn Thị Hài": 1.16, "Nguyễn Thị Hồng": 1.48
}
workers = list(proficiency_levels.keys())

#Hàm tính chi phí
def calculate_fitness_with_product(individual, tasks_times, proficiency_levels, df):
    detailed_task_allocation = {worker: [] for worker in workers}

    for task_idx, worker_list in enumerate(individual):
        task_time = tasks_times[task_idx]
        n_workers = len(worker_list)

        theoretical_time_per_worker = task_time / n_workers
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency
            detailed_task_allocation[worker].append(worker_time)

    worker_time_summary = [
        sum(worker_times) for worker_times in detailed_task_allocation.values()
    ]
    max_worker_time = max(worker_time_summary)

    stage_times = []
    for task_idx, worker_list in enumerate(individual):
        task_time = tasks_times[task_idx]
        n_workers = len(worker_list)
        theoretical_time_per_worker = task_time / n_workers
        worker_times = [theoretical_time_per_worker / proficiency_levels[worker] for worker in worker_list]

        if n_workers == 1:
            total_time_for_task = worker_times[0]
        elif n_workers == 2:
            total_time_for_task = min(worker_times) * 2
        elif n_workers == 3:
            total_time_for_task = min(worker_times) * 3

        stage_times.append(total_time_for_task)

    total_1 = sum(stage_times[0:4])
    total_2 = sum(stage_times[4:27])
    total_3 = sum(stage_times[27:50])
    total_4 = sum(stage_times[50:53])
    total_5 = sum(stage_times[53:57])
    total_6 = sum(stage_times[57:85])

    re_stage_time = total_1 + max(max(total_2, total_3) + total_4, total_5) + total_6

    total_product = (28800 - re_stage_time) / max_worker_time + 1
    fitness = -total_product
    return fitness, total_product

#hàm tính lương
def calculate_total_salary_with_constraints(individual, proficiency_levels, df):
    worker_salaries = {worker: 0 for worker in proficiency_levels.keys()}
    for task_idx, worker_list in enumerate(individual):
        task_price = df['ĐƠN GIÁ'][task_idx]
        num_workers = len(worker_list)
        if num_workers == 0:
            continue
        price_per_worker = task_price / num_workers
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_salaries[worker] += price_per_worker * proficiency
    total_salary = sum(worker_salaries.values())
    return total_salary, total_salary

# Hàm kiểm tra có thỏa ràng buộc hay không
def check_worker_machines(individual, machines):
    worker_machines = {worker: set() for worker in workers}
    for task_idx, worker_list in enumerate(individual):
        machine_type = machines[task_idx]
        for worker in worker_list:
            worker_machines[worker].add(machine_type)
    for i in list(worker_machines):
      if str(worker_machines[i]).replace("0,","").count(",") >= 2:
        return False
    return True

def is_valid_machine_assignment(worker, machine_type, worker_machine_assignment):
    assigned_machines = worker_machine_assignment[worker]
    non_zero_machines = {m for m in assigned_machines if m != 0}

    if machine_type == 0:
        return len(non_zero_machines) <= 2
    else:
        return len(non_zero_machines) < 2 or machine_type in assigned_machines

#Hàm trả về thiết bị công nhân sử dụng
def calculate_worker_machines(individual, machines):
    worker_machines = {worker: set() for worker in workers}
    for task_idx, worker_list in enumerate(individual):
        machine_type = machines[task_idx]
        for worker in worker_list:
            worker_machines[worker].add(machine_type)
    worker_machines_df = pd.DataFrame([
        {"Worker": worker, "Machines Used": sorted(list(machines_used))}
        for worker, machines_used in worker_machines.items()
    ]).sort_values(by="Worker").reset_index(drop=True)
    return worker_machines_df

# Hàm khởi tạo phân công công nhân
def greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df):
    sorted_workers = sorted(proficiency_levels.keys(), key=lambda x: proficiency_levels[x])
    individual = []
    worker_machine_assignment = {worker: set() for worker in sorted_workers}

    for task_idx, task_time in enumerate(tasks_times):
        machine_type = machines[task_idx]
        available_workers = [
            worker for worker in sorted_workers
            if is_valid_machine_assignment(worker, machine_type, worker_machine_assignment)
        ]

        if available_workers:
            n_selected_workers = random.randint(1, min(3, len(available_workers)))
            selected_workers = random.sample(available_workers, n_selected_workers)
            for worker in selected_workers:
                worker_machine_assignment[worker].add(machine_type)
            individual.append(selected_workers)
        else:
            # Nếu không còn công nhân hợp lệ, gán bất kỳ công nhân nào (xảy ra khi bị thiếu ràng buộc)
            fallback_worker = random.choice(sorted_workers)
            individual.append([fallback_worker])
            worker_machine_assignment[fallback_worker].add(machine_type)

    return individual

# Chạy thuật toán mô phỏng luyện kim
def simulated_annealing_with_constraints(
    initial_solution, tasks_times, proficiency_levels, df, initial_temp=1000, cooling_rate=0.95, max_iter=100000
):
    worker_machine_assignment = {worker: set() for worker in workers}

    current_solution = initial_solution
    current_fitness, current_product = calculate_total_salary_with_constraints(current_solution, proficiency_levels, df)
    best_solution = current_solution
    best_fitness = current_fitness
    best_product = current_product
    temperature = initial_temp

    for iteration in range(max_iter):
        new_solution = current_solution[:]
        task_idx = random.randint(0, len(tasks_times) - 1)
        machine_type = machines[task_idx]

        # Lựa chọn công nhân phù hợp
        possible_workers = [
            worker for worker in workers
            if is_valid_machine_assignment(worker, machine_type, worker_machine_assignment)
        ]
        n_selected_workers = random.randint(1, min(3, len(possible_workers))) if possible_workers else 1
        new_worker_list = random.sample(possible_workers, n_selected_workers) if possible_workers else [random.choice(workers)]

        # Gán giải pháp mới
        new_solution[task_idx] = new_worker_list
        for worker in new_worker_list:
            worker_machine_assignment[worker].add(machine_type)
        if check_worker_machines(new_solution, machines) == False:
            continue
        # Tính fitness cho giải pháp mới
        new_fitness, new_product = calculate_total_salary_with_constraints(new_solution, proficiency_levels, df)
        delta_fitness = new_fitness - current_fitness

        # Chấp nhận giải pháp mới
        if delta_fitness < 0 or np.exp(-delta_fitness / max(temperature, 1e-6)) > random.random():
            current_solution = new_solution
            current_fitness = new_fitness
            current_product = new_product

        # Cập nhật giải pháp tốt nhất
        if current_fitness < best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness
            best_product = current_product
            print(best_product)
        # Giảm nhiệt độ
        temperature *= cooling_rate

    return best_solution, -best_fitness, best_product

# Khởi tạo giải pháp ban đầu
initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)
while check_worker_machines(initial_solution, machines)==False:
  initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)

# Thực hiện tối ưu bằng thuatah toán mô phỏng luyện kim
best_solution, best_fitness, best_product = simulated_annealing_with_constraints(
    initial_solution, tasks_times, proficiency_levels, df
)

print("lương thấp nhất đạt được:", -best_fitness)

15449.853333333329
15400.893333333328
15388.119999999999
15292.6
15283.959999999995
15264.173333333329
15262.946666666663
15261.879999999997
15238.786666666661
15213.346666666663
15210.306666666665
15169.506666666663
15169.50666666666
15141.159999999994
15135.719999999996
15090.119999999999
15082.119999999999
15074.759999999997
15030.119999999997
15026.919999999996
15023.799999999997
15014.73333333333
15000.173333333329
14982.999999999998
14981.506666666666
14949.506666666664
14933.826666666666
14919.266666666666
14904.706666666665
14896.546666666665
14895.266666666665
14863.693333333333
14845.56
14845.559999999998
14833.799999999997
14826.439999999999
14826.439999999997
14825.373333333333
14821.426666666666
14821.426666666664
14819.026666666665
14755.346666666665
14748.946666666663
14724.41333333333
14723.879999999997
14723.879999999996
14621.799999999997
14620.733333333334
14620.733333333332
14620.73333333333
14610.386666666662
14563.986666666664
14556.626666666665
14555.346666666668

In [ ]:
initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)
while check_worker_machines(initial_solution, machines)==False:
  initial_solution = greedy_worker_assignment_with_constraints(tasks_times, machines, proficiency_levels, df)

# Optimize using Simulated Annealing with constraints
best_solution, best_fitness, best_product = simulated_annealing_with_constraints(
    initial_solution, tasks_times, proficiency_levels, df
)

print("lương thấp nhất đạt được:", -best_fitness)

15074.92
15062.919999999998
15060.786666666663
15051.266666666666
15035.826666666666
15033.053333333335
15025.159999999998
15016.626666666665
14984.786666666665
14978.44
14978.439999999997
14959.986666666666
14959.986666666664
14955.986666666666
14941.853333333334
14939.293333333335
14937.293333333335
14910.093333333334
14900.120000000004
14896.920000000002
14881.240000000002
14868.440000000002
14832.920000000004
14824.280000000002
14824.279999999999
14792.279999999999
14792.279999999997
14784.439999999997
14752.6
14707.000000000002
14707.0
14706.999999999998
14688.44
14688.439999999999
14674.626666666665
14671.426666666664
14609.56
14609.559999999996
14570.519999999997
14563.719999999996
14550.439999999999
14542.599999999997
14505.479999999996
14503.239999999998
14489.159999999998
14469.373333333331
14469.37333333333
14461.533333333333
14432.066666666666
14430.786666666667
14423.800000000001
14414.093333333334
14413.693333333336
14386.066666666671
14386.06666666667
14386.066666666668


In [ ]:
calculate_worker_machines(best_solution, machines)

,Worker,Machines Used
0,Bùi Thị Hồng Vân,"[0, 3, 6]"
1,Hoàng Thị Mỹ Yến,"[4, 5]"
2,Hoàng Thị Nở,[]
3,Hồ Thị Diệp,[]
4,Lê Thị Như Quỳnh,[]
5,Lê Thị Phương Thảo,[]
6,Lê Thị Thanh Hà,[]
7,Nguyễn Thị Gấm,[]
8,Nguyễn Thị Hài,[4]
9,Nguyễn Thị Hồng,[]


In [ ]:
# Hàm tính lương và tổng tiền lương cho tất cả công nhân
def calculate_salary(individual, proficiency_levels, dt):
    worker_salaries = {worker: 0 for worker in workers}  # Khởi tạo lương cho mỗi công nhân bằng 0

    for task_idx, worker_list in enumerate(individual):
        # Lấy đơn giá của công đoạn hiện tại
        task_price = dt['ĐƠN GIÁ'][task_idx]
        num_workers = len(worker_list)  # Số công nhân thực hiện công đoạn

        # Chia đơn giá cho số công nhân nếu nhiều người thực hiện
        price_per_worker = task_price / num_workers

        # Tính lương cho từng công nhân tham gia công đoạn
        for worker in worker_list:
            proficiency = proficiency_levels[worker]  # Lấy năng lực của công nhân
            # Lương cho công đoạn này bằng đơn giá chia cho số công nhân, sau đó nhân năng lực
            salary_for_task = price_per_worker * proficiency
            # Cộng lương cho công đoạn này vào tổng lương của công nhân
            worker_salaries[worker] += salary_for_task

    # Tạo DataFrame chứa kết quả lương của từng công nhân
    salary_df = pd.DataFrame.from_dict(worker_salaries, orient='index', columns=['Tiền Lương'])
    salary_df.reset_index(inplace=True)
    salary_df.columns = ['Công Nhân', 'Tổng Tiền Lương']

    # Tính tổng tiền lương của tất cả công nhân
    total_salary = salary_df['Tổng Tiền Lương'].sum()
    print(f"Tổng tiền lương của tất cả công nhân: {total_salary:.2f}")

    return salary_df, total_salary
# Sử dụng hàm để tính lương cho mỗi công nhân và tổng lương
salary_df, total_salary = calculate_salary(best_solution, proficiency_levels, df)

# Hiển thị DataFrame chứa tổng tiền lương cho mỗi công nhân
print(salary_df)
print(f"Tổng tiền lương của tất cả công nhân: {total_salary:.2f}")


Tổng tiền lương của tất cả công nhân: 12980.52
              Công Nhân  Tổng Tiền Lương
0    Lê Thị Phương Thảo         0.000000
1      Hoàng Thị Mỹ Yến       338.333333
2        Trần Thị Tuyền         0.000000
3         Ngô Khắc Hiển      4254.166667
4           Hồ Thị Diệp         0.000000
5   Nguyễn Thị Mỹ Nhung         0.000000
6          Thái Thị Nga       482.560000
7          Trần Thị Cúc      6309.666667
8       Lê Thị Thanh Hà         0.000000
9       Nguyễn Thị Thúy         0.000000
10     Trần Thị Cẩm Rin         0.000000
11         Hoàng Thị Nở         0.000000
12     Lê Thị Như Quỳnh         0.000000
13      Trần Thị Thương        58.773333
14          Đinh Thị Mi         0.000000
15  Nguyễn Thị Ngọc Lan         0.000000
16     Bùi Thị Hồng Vân       883.166667
17        Trần Thị Oanh         0.000000
18       Nguyễn Thị Gấm         0.000000
19    Phan Thị Thu Hoài         0.000000
20       Nguyễn Thị Thu         0.000000
21       Nguyễn Thị Hài       653.853333
22      Ng

In [ ]:
salary_df

,Công Nhân,Tổng Tiền Lương
0,Lê Thị Phương Thảo,0.000000
1,Hoàng Thị Mỹ Yến,338.333333
2,Trần Thị Tuyền,0.000000
3,Ngô Khắc Hiển,4254.166667
4,Hồ Thị Diệp,0.000000
5,Nguyễn Thị Mỹ Nhung,0.000000
6,Thái Thị Nga,482.560000
7,Trần Thị Cúc,6309.666667
8,Lê Thị Thanh Hà,0.000000
9,Nguyễn Thị Thúy,0.000000


In [ ]:
def display_worker_tasks(individual):
    # Tạo một dictionary lưu trữ các công đoạn mà mỗi công nhân đảm nhiệm
    worker_tasks = {worker: [] for worker in workers}

    # Lặp qua từng công đoạn trong cá thể tốt nhất
    for task_idx, worker_list in enumerate(individual):
        # Lấy tên công đoạn và loại máy cho công đoạn hiện tại
        task_name = df['TÊN CÔNG ĐOẠN'].iloc[task_idx]  # Sử dụng iloc để tránh lỗi index
        machine_type = machines[task_idx]

        # Thêm công đoạn và loại máy vào danh sách công việc của từng công nhân
        for worker in worker_list:
            worker_tasks[worker].append(f"{task_name} (Máy {machine_type})")

    # Hiển thị kết quả
    for worker, tasks in worker_tasks.items():
        if tasks:  # Nếu công nhân có công việc
            tasks_str = ", ".join(tasks)
            print(f"Công nhân {worker} làm các công đoạn: {tasks_str}")
        else:
            print(f"Công nhân {worker} không có công việc.")

display_worker_tasks(best_solution)

Công nhân Lê Thị Phương Thảo không có công việc.
Công nhân Hoàng Thị Mỹ Yến làm các công đoạn: Kansai đáy thân sau - 4 (Máy 4), Ép keo paget - 5 (Máy 5), Chạy kansai lưng - 4 (Máy 4)
Công nhân Trần Thị Tuyền không có công việc.
Công nhân Ngô Khắc Hiển làm các công đoạn: Xã phà thun lưng - 0 (Máy 0), Đo cắt pat túi thân sau - 0 (Máy 0), Ép dây pat túi thân sau - 2 (Máy 2), Gọt lộn dây treo túi sau + lộn - 0 (Máy 0), Ép keo cơi túi - 2 (Máy 2), Bỏ rập đh cơi túi ( xù, pat, lược tp cơi) - 7 (Máy 7), Bỏ rập dh túi thân sau + lụa số - 7 (Máy 7), Vs3c bao túi thân sau + lộn - 2 (Máy 2), Lượ số vs5c decuop vào túi thân sau - 2 (Máy 2), Lựa số + vss5c đecup thân sau - 2 (Máy 2), Lựa số + vs4c đáy thân sau+ canh sọc - 2 (Máy 2), Vs3c xq túi đồng hồ - 2 (Máy 2), Bỏ rập đh túi hông x2 + lấy dấu + canh sọc - 7 (Máy 7), Vs3c bao túi hông x2+lộn - 2 (Máy 2), Vs3c paget tròn - 2 (Máy 2), Bấm góc vs3c đáy tt - 2 (Máy 2), Lựa số+ bỏ rập đh tt + lấy dấu + paget - 7 (Máy 7), May nhãn logo thân trước + lấ

In [ ]:
df['Công Nhân Mới (Nhiều)'] = [", ".join(worker_list) for worker_list in best_solution]

# Hiển thị kết quả cuối cùng
df[['TÊN CÔNG ĐOẠN', 'Công Nhân Mới (Nhiều)']]

,TÊN CÔNG ĐOẠN,Công Nhân Mới (Nhiều)
0,Xã phà thun lưng - 0,Ngô Khắc Hiển
1,Lấy dấu cắt gai xù túi sau + gọt góc - 0,Bùi Thị Hồng Vân
2,Đo cắt pat túi thân sau - 0,"Trần Thị Cúc, Ngô Khắc Hiển"
3,Đo cắt dây treo gắn túi thân sau - 0,Bùi Thị Hồng Vân
4,Nối dây pat túi thân sau - 1,Trần Thị Cúc
...,...,...
80,Vệ sinh công nghiệp - 0,Trần Thị Cúc
81,Lộn quần ( trái sang phải ) - 0,Ngô Khắc Hiển
82,Mí đáy thân trước - 1,Trần Thị Cúc
83,Lấy dấu lưng thùa khuyu - 0,Bùi Thị Hồng Vân


In [ ]:
def display_task_allocation_with_time(individual):
    detailed_task_allocation = {worker: [] for worker in workers}

    for task_idx, worker_list in enumerate(individual):
        task_time = df['TGLT'][task_idx]
        n_workers = len(worker_list)

        # Tính thời gian chia đều lý thuyết cho mỗi công nhân
        theoretical_time_per_worker = task_time / n_workers

        # Điều chỉnh thời gian cho từng công nhân theo độ thông thạo của họ
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency  # Điều chỉnh thời gian theo độ thông thạo
            detailed_task_allocation[worker].append(f"Công đoạn {df['TÊN CÔNG ĐOẠN'].iloc[task_idx]}: {worker_time:.2f} giây")

    return detailed_task_allocation


# Chọn cá thể tốt nhất (cá thể có độ lệch chuẩn nhỏ nhất)
task_allocation_with_time = display_task_allocation_with_time(best_solution)

# Hiển thị chi tiết công việc cho mỗi công nhân
worker_task_summary = []
worker_time = []

for worker, tasks in task_allocation_with_time.items():
    # Tính tổng thời gian cho mỗi công nhân
    total_time = sum(float(task.split(": ")[1].split()[0]) for task in tasks)
    # Chi tiết các công đoạn
    task_details = " + ".join(tasks)
    worker_task_summary.append({
        'Công Nhân': worker,
        'Chi Tiết Công Việc': task_details,
        'Tổng Thời Gian': total_time
    })
    worker_time.append({
        'Công Nhân': worker,
        'Tổng Thời Gian': total_time
    })

# Chuyển thành DataFrame để hiển thị
worker_task_summary_df = pd.DataFrame(worker_task_summary)

# Hiển thị kết quả chi tiết cho mỗi công nhân
worker_task_summary_df
def display_task_allocation_with_time(individual):
    detailed_task_allocation = {worker: [] for worker in workers}

    for task_idx, worker_list in enumerate(individual):
        task_time = df['TGLT'][task_idx]
        n_workers = len(worker_list)

        # Tính thời gian chia đều lý thuyết cho mỗi công nhân
        theoretical_time_per_worker = task_time / n_workers

        # Điều chỉnh thời gian cho từng công nhân theo độ thông thạo của họ
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency  # Điều chỉnh thời gian theo độ thông thạo
            detailed_task_allocation[worker].append(f"Công đoạn {df['TÊN CÔNG ĐOẠN'].iloc[task_idx]}: {worker_time:.2f} giây")

    return detailed_task_allocation


# Chọn cá thể tốt nhất (cá thể có độ lệch chuẩn nhỏ nhất)
task_allocation_with_time = display_task_allocation_with_time(best_solution)

# Hiển thị chi tiết công việc cho mỗi công nhân
worker_task_summary = []
worker_time = []

for worker, tasks in task_allocation_with_time.items():
    # Tính tổng thời gian cho mỗi công nhân
    total_time = sum(float(task.split(": ")[1].split()[0]) for task in tasks)
    # Chi tiết các công đoạn
    task_details = " + ".join(tasks)
    worker_task_summary.append({
        'Công Nhân': worker,
        'Chi Tiết Công Việc': task_details,
        'Tổng Thời Gian': total_time
    })
    worker_time.append({
        'Công Nhân': worker,
        'Tổng Thời Gian': total_time
    })

# Chuyển thành DataFrame để hiển thị
worker_task_summary_df = pd.DataFrame(worker_task_summary)

# Hiển thị kết quả chi tiết cho mỗi công nhân
worker_task_summary_df


,Công Nhân,Chi Tiết Công Việc,Tổng Thời Gian
0,Lê Thị Phương Thảo,,0.00
1,Hoàng Thị Mỹ Yến,Công đoạn Kansai đáy thân sau - 4: 8.33 giây +...,47.12
2,Trần Thị Tuyền,,0.00
3,Ngô Khắc Hiển,Công đoạn Xã phà thun lưng - 0: 3.00 giây + Cô...,802.00
4,Hồ Thị Diệp,,0.00
5,Nguyễn Thị Mỹ Nhung,,0.00
6,Thái Thị Nga,Công đoạn diễu kan sai đáy trong - 4: 34.48 gi...,68.10
7,Trần Thị Cúc,Công đoạn Đo cắt pat túi thân sau - 0: 1.00 gi...,1194.00
8,Lê Thị Thanh Hà,,0.00
9,Nguyễn Thị Thúy,,0.00


In [ ]:
best_solution

[['Ngô Khắc Hiển'],
 ['Bùi Thị Hồng Vân'],
 ['Trần Thị Cúc', 'Ngô Khắc Hiển'],
 ['Bùi Thị Hồng Vân'],
 ['Trần Thị Cúc'],
 ['Ngô Khắc Hiển'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Ngô Khắc Hiển'],
 ['Ngô Khắc Hiển'],
 ['Ngô Khắc Hiển'],
 ['Trần Thị Cúc'],
 ['Ngô Khắc Hiển'],
 ['Bùi Thị Hồng Vân'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Ngô Khắc Hiển'],
 ['Trần Thị Cúc'],
 ['Ngô Khắc Hiển'],
 ['Ngô Khắc Hiển'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc', 'Bùi Thị Hồng Vân'],
 ['Ngô Khắc Hiển'],
 ['Hoàng Thị Mỹ Yến', 'Trần Thị Thương', 'Nguyễn Thị Hài'],
 ['Trần Thị Cúc'],
 ['Hoàng Thị Mỹ Yến'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Ngô Khắc Hiển'],
 ['Ngô Khắc Hiển'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Ngô Khắc Hiển'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Trần Thị Cúc'],
 ['Ngô Khắc Hiển'],
 ['Ngô Khắc Hiển'],
 ['Ngô Khắc Hiển'],
 ['Trần Thị Cúc'

In [ ]:
def display_stage_times(individual):
    stage_times = []

    for task_idx, worker_list in enumerate(individual):
        task_time = df['TGLT'][task_idx]
        n_workers = len(worker_list)

        # Tính thời gian lý thuyết chia đều cho mỗi công nhân
        theoretical_time_per_worker = task_time / n_workers

        # Tính thời gian điều chỉnh cho mỗi công nhân dựa trên độ thông thạo
        worker_times = []
        for worker in worker_list:
            proficiency = proficiency_levels[worker]
            worker_time = theoretical_time_per_worker / proficiency  # Điều chỉnh thời gian theo độ thông thạo
            worker_times.append(worker_time)

        # Nếu có 1 công nhân, giữ nguyên thời gian
        if n_workers == 1:
            total_time_for_task = worker_times[0]
        # Nếu có 2 công nhân, lấy người nhanh nhất và nhân đôi thời gian
        elif n_workers == 2:
            total_time_for_task = min(worker_times) * 2
        # Nếu có 3 công nhân, lấy người nhanh nhất và nhân ba thời gian
        elif n_workers == 3:
            total_time_for_task = min(worker_times) * 3

        # Lưu trữ thời gian cho từng công đoạn
        stage_times.append({
            'Công Đoạn': df['TÊN CÔNG ĐOẠN'].iloc[task_idx],
            'Tổng Thời Gian': total_time_for_task
        })

    return stage_times

# Sử dụng hàm display_stage_times để hiển thị thời gian của từng công đoạn
stage_times_summary = display_stage_times(best_solution)

# Chuyển thành DataFrame để hiển thị
stage_times_df = pd.DataFrame(stage_times_summary)

# Hiển thị kết quả
stage_times_df

,Công Đoạn,Tổng Thời Gian
0,Xã phà thun lưng - 0,3.0
1,Lấy dấu cắt gai xù túi sau + gọt góc - 0,4.0
2,Đo cắt pat túi thân sau - 0,2.0
3,Đo cắt dây treo gắn túi thân sau - 0,5.0
4,Nối dây pat túi thân sau - 1,10.0
...,...,...
80,Vệ sinh công nghiệp - 0,31.0
81,Lộn quần ( trái sang phải ) - 0,5.0
82,Mí đáy thân trước - 1,15.0
83,Lấy dấu lưng thùa khuyu - 0,9.0
